In [3]:
import pandas as pd 
from datetime import datetime
import openpyxl as px 
import psycopg2 as pg
import configparser
import numpy as np
import os

In [4]:
def mascara(df, coluna,n_digitos):
    df[coluna] = df[coluna].astype(str)
    df[coluna] = str("000") + df[coluna]
    df[coluna] = df[coluna].astype(str).str[-n_digitos:] 
    return df

In [5]:
path_justificativas = r'F:\Qualidade_Florestal\01- SÃO PAULO\07- Monitoramentos de Silvicultura\06 - SOP CODE\04 -Bases de Configuração\Excel_py\justificativas.xlsx'

In [6]:
path_retalhonamento = r'\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento_e_Controle\Geoprocessamento\02_SETOR\07_CADASTRO\14_BASE RETALHONAMENTOS\DExPARA\SHP'

In [7]:
retalhonamentos_justificado = pd.read_excel(path_justificativas , sheet_name='retalhonamento', dtype={'objeto de locação origem':'object', 'objeto de locação retalhonado':'object'})

In [8]:
arquivos_retalhonamentos = []



# Caminhar pela árvore de diretórios
for root, dirs, files in os.walk(path_retalhonamento):
    for a in files:
        if a.endswith('.xlsx') and 'de_para' in a.lower() and '~' not in a and not 'parcial' in a.lower() :
            file_path = os.path.join(root, a)
            creation_time = os.path.getctime(file_path)
            creation_date = datetime.fromtimestamp(creation_time).strftime('%Y-%m-%d')
            origem = os.path.basename(root)
            nome_do_arquivo = a
            arquivos_retalhonamentos.append((file_path, a, creation_date, origem))
            


colunas_ret = ['Arquivo', 'nome do arquivo', 'data','diretorio']
links_retalhonamento = pd.DataFrame(arquivos_retalhonamentos, columns=colunas_ret)
links_retalhonamento = links_retalhonamento.sort_values(['data'], ascending=False).drop_duplicates(['Arquivo'], keep='first')



retalhonamentos = pd.DataFrame()
for _, n in links_retalhonamento.iterrows():
    base = pd.read_excel(n['Arquivo'],skiprows=1)
    base['nome_do_arquivo'] = n['nome do arquivo']
    retalhonamentos = pd.concat([retalhonamentos,base], ignore_index=True)

In [9]:
#Retalhonamento Justificado Inclusão e exclusão

retalhonamentos_justificado = retalhonamentos_justificado[retalhonamentos_justificado['status retalhonamento'] == 'Incluir']


retalhonamentos_justificado = mascara(retalhonamentos_justificado,'objeto de locação origem',7)

retalhonamentos_justificado = mascara(retalhonamentos_justificado,'objeto de locação retalhonado',7)

#Definindo ID_PROJETO
retalhonamentos_justificado['ID_PROJETO'] = retalhonamentos_justificado['objeto de locação origem'].str[:4]

#Definindo ID_PROJETO_1
retalhonamentos_justificado['ID_PROJETO_1'] = retalhonamentos_justificado['objeto de locação retalhonado'].str[:4]

#Definindo ID_TALHÃO
retalhonamentos_justificado['ID_TALHAO'] = retalhonamentos_justificado['objeto de locação origem'].str[-3:]

#Definindo ID_TALHÃO_1
retalhonamentos_justificado['ID_TALHAO_1'] = retalhonamentos_justificado['objeto de locação retalhonado'].str[-3:]

#Definindo as datas de retalhonamento, apenas copiando a primeira data

retalhonamentos_justificado['DATA_CRIAC'] = retalhonamentos_justificado['Data de retalhonamento']

retalhonamentos_justificado['AREA_HA'] = 9999


ret_rename = {
    'cd uso solo origem':'CD_USO_SOL',
    'cd uso solo retalhonado': 'CD_USO_SOL_1',
    'Data de retalhonamento': 'DATA_CRIAC_1'
    
    }
retalhonamentos_justificado = retalhonamentos_justificado.rename(ret_rename,axis=1)

#Alterar o tipo de dados para ficar igual a da base retalhonamentos e não ter problemas futuros no código

ret_jus_type = {'CD_USO_SOL':'float64',
                'CD_USO_SOL_1':'float64',
                'objeto de locação origem':'object',
                'objeto de locação retalhonado':'object',
                'DATA_CRIAC':'datetime64[ns]',
                'DATA_CRIAC_1':'datetime64[ns]'
                }

retalhonamentos_justificado = retalhonamentos_justificado.astype(ret_jus_type)



#Criar uma base para exclusão de retalhonamentos
#Essa base só precisa de cd_uso_origem para excluir a informação da base original

retalhonamentos_justificado_excluir = pd.read_excel(path_justificativas, sheet_name='retalhonamento', dtype={'cd uso solo origem':'int64'})

retalhonamentos_justificado_excluir = mascara(retalhonamentos_justificado_excluir,'objeto de locação origem',7)

retalhonamentos_justificado_excluir = retalhonamentos_justificado_excluir[retalhonamentos_justificado_excluir['status retalhonamento'] == 'Excluir']

In [ ]:
#Escolher colunas para importar base de retalhonamento


retalhonamentos = retalhonamentos[['CD_USO_SOL','ID_PROJETO', 'ID_TALHAO','AREA_HA', 'DATA_CRIAC', 'CD_USO_SOL_1','ID_PROJETO_1', 'ID_TALHAO_1', 'DATA_CRIAC_1']]


#Criando coluna de apoio para retalhonamentos

#Definindo o formato de dados dentro a base de retalhonamentos
retalhonamentos = mascara(retalhonamentos,'ID_PROJETO',4)
retalhonamentos = mascara(retalhonamentos,'ID_TALHAO',3)
retalhonamentos['objeto de locação origem'] = retalhonamentos['ID_PROJETO'] + retalhonamentos['ID_TALHAO']
retalhonamentos = mascara(retalhonamentos,'objeto de locação origem',7)

#Preenchendo campos vazios com zero para transformar em str
retalhonamentos['ID_PROJETO_1'] = retalhonamentos['ID_PROJETO_1'].fillna(0).astype(int).astype(str)
retalhonamentos['ID_TALHAO_1'] = retalhonamentos['ID_TALHAO_1'].fillna(0).astype(int).astype(str)

#Definindo o formato de dados dentro a base de retalhonamentos
retalhonamentos = mascara(retalhonamentos,'ID_PROJETO_1',4)
retalhonamentos = mascara(retalhonamentos,'ID_TALHAO_1',3)
retalhonamentos['objeto de locação retalhonado'] = retalhonamentos['ID_PROJETO_1'] + retalhonamentos['ID_TALHAO_1']
retalhonamentos = mascara(retalhonamentos,'objeto de locação retalhonado',7)


#Exportar o retalhonamento

#retalhonamentos.to_excel('Retalhonamento Geo.xlsx')



#Inserir exclusão de apontamentos

retalhonamentos = retalhonamentos.merge(retalhonamentos_justificado_excluir[['cd uso solo origem', 'status retalhonamento']], left_on='CD_USO_SOL', right_on='cd uso solo origem', how='left').drop('cd uso solo origem',axis=1)

#Filtrando tudo que for diferente de excluir, garantindo que os retalhonamento excluídos não estejam na base
retalhonamentos = retalhonamentos[retalhonamentos['status retalhonamento'] != 'Excluir']

#Inserir retalhonamentos justificados

retalhonamentos = pd.concat([retalhonamentos,retalhonamentos_justificado])



# Alterando o tipo de dado para retalhonamento
retalhonamentos['CD_USO_SOL'] = retalhonamentos['CD_USO_SOL'].fillna(0).astype(int)
retalhonamentos['CD_USO_SOL_1'] = retalhonamentos['CD_USO_SOL_1'].fillna(0).astype(int)



# Definindo o tipo de dado
dtype_retalhonamento = {'DATA_CRIAC':'datetime64[ns]',
                         'DATA_CRIAC_1':'datetime64[ns]',
                         'CD_USO_SOL':'object',
                         'CD_USO_SOL_1':'object'}

retalhonamentos = retalhonamentos.astype(dtype_retalhonamento)


# Remover apontamentos anteriores a 2020

retalhonamentos = retalhonamentos[(retalhonamentos['DATA_CRIAC'].dt.year >= 2020) &  (retalhonamentos['DATA_CRIAC_1'].dt.year >= 2020)]


# Filtrando o que não tem cd_uso_solo definido
retalhonamentos = retalhonamentos[(pd.notna(retalhonamentos['CD_USO_SOL'])) | (pd.notna(retalhonamentos['CD_USO_SOL_1']) ) ]

# Encontrando o que temos de retalhonamento
# Precisamos que os talhões sejam diferentes tanto objeto de locação quanto o cd_uso_solo se qualquer um dos dois for, excluir
retalhonamentos['delta_cd_uso_sol'] = (retalhonamentos['CD_USO_SOL'] == retalhonamentos['CD_USO_SOL_1'])
retalhonamentos['delta_talhao'] = (retalhonamentos['objeto de locação origem'] == retalhonamentos['objeto de locação retalhonado'])


# Filtrando somente talhões que tem identificação de retalhonameto
retalhonamentos = retalhonamentos[retalhonamentos['delta_cd_uso_sol'] == False]
retalhonamentos = retalhonamentos[retalhonamentos['delta_talhao'] == False]


# Retirar talhões zero da base
retalhonamentos = retalhonamentos[retalhonamentos['CD_USO_SOL'] != 0]
retalhonamentos = retalhonamentos[retalhonamentos['CD_USO_SOL_1'] != 0]


# Remover talhões vindos de retalhonamentos, sempre mantendo o que tem a maior área
retalhonamentos = retalhonamentos.sort_values(['CD_USO_SOL_1', 'AREA_HA'], ascending=[False,False])
retalhonamentos = retalhonamentos.drop_duplicates('CD_USO_SOL_1', keep='first')


# Organizar talhões de forma ascendente
retalhonamentos = retalhonamentos.sort_values(['CD_USO_SOL','CD_USO_SOL_1', 'DATA_CRIAC_1'], ascending=[False,False, True])

retalhonamentos = retalhonamentos.rename({'CD_USO_SOL':'cd_uso_solo_origem', 
                                           'CD_USO_SOL_1':'cd_uso_solo_retalhonado',
                                           'DATA_CRIAC':'Data de criação origem',
                                           'DATA_CRIAC_1':'Data de criação retalhonado'
                                           },axis=1)


# Removendo onde temos duplicatas em todas as linhas
retalhonamentos = retalhonamentos.drop_duplicates(keep='first')


# Criar lógica para encontrar se temos talhões iguais, mas com datas distintas
retalhonamentos['erro cadastro'] = (
     (retalhonamentos['cd_uso_solo_origem'] == retalhonamentos['cd_uso_solo_origem'].shift(1)) &
         (retalhonamentos['cd_uso_solo_retalhonado'] == retalhonamentos['cd_uso_solo_retalhonado'].shift(1)) &
             (retalhonamentos['Data de criação retalhonado'] != retalhonamentos['Data de criação retalhonado'].shift(1))
  )

# Selecionando somente o que é igual, excluindo o que é True

retalhonamentos = retalhonamentos[retalhonamentos['erro cadastro'] != True ]


retalhonamentos.to_excel('retalhonamento ajustado.xlsx')